In [1]:
import numpy as np
import scipy.sparse
from scipy.sparse.linalg import lobpcg, eigsh, svds
from scipy.sparse import csr_matrix

In [2]:
adj_path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.adj"
adjacency_matrix = scipy.sparse.load_npz(adj_path + ".npz")

In [ ]:
window_size = 2

diags = np.asarray(adjacency_matrix.sum(axis=1)).flatten()
n = adjacency_matrix.shape[0]
D_inv = scipy.sparse.spdiags(1./diags, [0], n, n, format='csr')
DA = D_inv.dot(adjacency_matrix)
DA_r = D_inv.dot(adjacency_matrix)
deepwalk_operator = DA
for r in range(2, window_size+1):
    DA_r = DA_r.dot(DA)
    deepwalk_operator += DA_r
    print(r)
deepwalk_operator /= window_size
deepwalk_operator = deepwalk_operator.dot(D_inv)

In [ ]:
C = diags.sum() / 5.0
deepwalk_operator.data = np.log(C * deepwalk_operator.data)

In [ ]:
path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.skip-gram"
scipy.sparse.save_npz(path, deepwalk_operator)

In [ ]:
deep_vals, deep_vecs = eigsh(deepwalk_operator, k=1, which='LM')